# Exploration 14
# 미니프로젝트: Movielens 영화 추천 실습 🎞
|---평가문항---|---상세기준---|
|:------|:------|
|1. CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

## Movielens 데이터
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

# 1. 데이터 준비와 전처리

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남긴다.
ratings = ratings[ratings['rating'] >= 3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# 2. 데이터 분석
## 1) ratings에 있는 유니크한 영화 개수

In [5]:
print("영화 개수: ", ratings['movie_id'].nunique())

영화 개수:  3628


## 2) ratings에 있는 유니크한 사용자 수

In [6]:
print("유저 수: ", ratings['user_id'].nunique())

유저 수:  6039


## 3) 가장 인기 있는 영화 30개(인기순)

In [7]:
# movie_id를 기준으로 merge 하여 새로운 데이터 프레임 생성
merge_df = pd.merge(ratings, movies, on='movie_id')
merge_df.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [8]:
count_movies = merge_df.groupby('title')['user_id'].count()
count_movies = count_movies.sort_values(ascending=False)
count_movies.head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

# 3. 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [9]:
# 사용하는 컬럼만 남겨준다
cols = ['user_id', 'title', 'count']
movie_data = merge_df[cols]
movie_data.head()

,user_id,title,count
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5


In [10]:
# 내가 선호하는 영화 5개
my_fav = [
    'Star Wars: Episode VI - Return of the Jedi (1983)',
    'Men in Black (1997)',
    'Forrest Gump (1994)',
    'Toy Story (1995)',
    'Seven (1995)'
]

# 유저 id, 영화 제목, 평점
my_rating = pd.DataFrame({'user_id' : [6041] * 5, 'title' : my_fav, 'count' : [4, 5, 5, 5, 5]})
my_rating

,user_id,title,count
0,6041,Star Wars: Episode VI - Return of the Jedi (1983),4
1,6041,Men in Black (1997),5
2,6041,Forrest Gump (1994),5
3,6041,Toy Story (1995),5
4,6041,Seven (1995),5


In [11]:
# 내 점수를 뒤에 붙인다
movie_data = movie_data.append(my_rating)

In [12]:
movie_data.tail(10)

,user_id,title,count
836473,5851,One Little Indian (1973),5
836474,5854,Slaughterhouse (1987),4
836475,5854,"Promise, The (Versprechen, Das) (1994)",3
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4
836477,5948,Identification of a Woman (Identificazione di ...,5
0,6041,Star Wars: Episode VI - Return of the Jedi (1983),4
1,6041,Men in Black (1997),5
2,6041,Forrest Gump (1994),5
3,6041,Toy Story (1995),5
4,6041,Seven (1995),5


In [13]:
movie_data[movie_data['user_id'] == 6041]

,user_id,title,count
0,6041,Star Wars: Episode VI - Return of the Jedi (1983),4
1,6041,Men in Black (1997),5
2,6041,Forrest Gump (1994),5
3,6041,Toy Story (1995),5
4,6041,Seven (1995),5


## 4. CSR matrix를 직접 만들어 봅시다.

In [14]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = movie_data['user_id'].unique()
title_unique = movie_data['title'].unique()

# 유저, 아티스트 indexing 
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [15]:
# 인덱싱이 제대로 되었는지 확인
print(user_to_idx[6041])
print(title_to_idx['Back to the Future (1985)'])

6039
22


In [16]:
temp_user_data = movie_data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movie_data): # 모든 row가 정상적으로 인덱싱되었다면
    print("user_id column indexing OK!!")
    movie_data['user_id'] = temp_user_data # movie_data['user_id']을 인덱싱된 Series로 교체해준다
else:
    print("user_id column indexing Fail!!")

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱 해준다
temp_title_data = movie_data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(movie_data):
    print("title column indexing OK!!")
    movie_data['title'] = temp_title_data
else:
    print("title column indexing Fail!!")

movie_data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,count
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,64,4
1,6039,175,5
2,6039,160,5
3,6039,40,5


In [17]:
# CSR matrix 만들기
from scipy.sparse import csr_matrix

num_user = movie_data['user_id'].nunique()
num_movie = movie_data['title'].nunique()

csr_data = csr_matrix((movie_data['count'], (movie_data['user_id'], movie_data['title'])), shape=(num_user, num_movie))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [18]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [19]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [20]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose 해준다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [21]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

## 6. 훈련된 모델이 예측한 나의 선호도 파악
### 내가 선호하는 영화의 선호도 예측

In [22]:
my_id = user_to_idx[6041] # 내가 새로 추가한 유저 아이디
saving_ryan = title_to_idx['Toy Story (1995)']

my_vector, saving_ryan_vector = als_model.user_factors[my_id], als_model.item_factors[saving_ryan]
print("토이 스토리 선호도: ", np.dot(my_vector, saving_ryan_vector))

토이 스토리 선호도:  0.48538798


### 다른 영화에 대한 나의 선호도 예측

In [23]:
my_id = user_to_idx[6041] # 내가 새로 추가한 유저 아이디
saving_ryan = title_to_idx['Men in Black (1997)']

my_vector, saving_ryan_vector = als_model.user_factors[my_id], als_model.item_factors[saving_ryan]
print("맨인블랙 선호도: ", np.dot(my_vector, saving_ryan_vector))

맨인블랙 선호도:  0.46142977


## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [24]:
idx_to_title = {v:k for k, v in title_to_idx.items()} # 영화 이름 : 영화 idx

def get_similar_movie(movie_title: str):
    title_id = title_to_idx[movie_title]
    similar_movie = als_model.similar_items(title_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [25]:
get_similar_movie('Forrest Gump (1994)')

['Forrest Gump (1994)',
 'Seven (1995)',
 'Groundhog Day (1993)',
 'Pretty Woman (1990)',
 'As Good As It Gets (1997)',
 'Ghost (1990)',
 'Sleepless in Seattle (1993)',
 'Clueless (1995)',
 'Four Weddings and a Funeral (1994)',
 "My Best Friend's Wedding (1997)"]

In [26]:
get_similar_movie('Star Wars: Episode VI - Return of the Jedi (1983)')

['Star Wars: Episode VI - Return of the Jedi (1983)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Seven (1995)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Raiders of the Lost Ark (1981)',
 'Back to the Future (1985)',
 'Forrest Gump (1994)',
 'Open Season (1996)',
 'Indiana Jones and the Last Crusade (1989)']

## 7-1. 내가 가장 좋아할 만한 영화들을 추천받기

In [27]:
movie_recommended = als_model.recommend(my_id, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.4560038),
 (117, 0.42675376),
 (60, 0.3825466),
 (110, 0.37398186),
 (107, 0.37157542),
 (44, 0.3637982),
 (4, 0.3365166),
 (124, 0.29207698),
 (92, 0.279186),
 (99, 0.23671934),
 (322, 0.23060228),
 (22, 0.22782841),
 (33, 0.22362313),
 (150, 0.22337313),
 (670, 0.21393132),
 (48, 0.19125037),
 (475, 0.18990564),
 (38, 0.18773425),
 (204, 0.17651957),
 (20, 0.17616828)]

In [28]:
[idx_to_title[i[0]] for i in movie_recommended]

['Toy Story 2 (1999)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Groundhog Day (1993)',
 'Jurassic Park (1993)',
 'Star Wars: Episode IV - A New Hope (1977)',
 "Bug's Life, A (1998)",
 'Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'American Beauty (1999)',
 'Babe (1995)',
 'Back to the Future (1985)',
 'Aladdin (1992)',
 'Independence Day (ID4) (1996)',
 'Galaxy Quest (1999)',
 'Saving Private Ryan (1998)',
 'My Cousin Vinny (1992)',
 'Sixth Sense, The (1999)',
 'Clueless (1995)',
 'Pleasantville (1998)']

In [29]:
brave = title_to_idx['Star Wars: Episode IV - A New Hope (1977)']
explain = als_model.explain(my_id, csr_data, itemid=brave)

In [30]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Star Wars: Episode VI - Return of the Jedi (1983)', 0.3508879187385134),
 ('Men in Black (1997)', 0.020860648940917905),
 ('Toy Story (1995)', 0.003418042465602244),
 ('Seven (1995)', 0.0006187175171926824),
 ('Forrest Gump (1994)', -0.015879148323293887)]

# 회고
- 유튜브, 넷플릭스, 인터넷 광고 등 여러 플랫폼에서 사용하는 추천 시스템에 대해서 궁금한 부분이 많았었다.(마침 내가 궁금했던걸 추천해줄때 진짜 소름;;)
- 이번 노드를 통해서 추천 시스템에 대한 어느정도의 궁금증을 해결한 것 같다. 
- 그러나 노드를 진행하면서 의문이 들었다. 선호하는 것이 명확한 사람도 있지만, 가령 애니메이션을 좋아하면서 호러를 좋아하는 사람들도 존재한다. 그런 사람들에게 추천 시스템이 잘 적용할까?
- 나름 재미있기도 했지만 어렵고, 여러 의문이 생기는 노드였다. 